In [1]:
import pandas as pd
import os,pickle
import numpy as np
from collections import Counter
from multiprocessing import Pool

In [2]:
out_dir = './figure5'

In [22]:
### Read synonymous candidate list and create pivot table for genes
cohorts = ['histology','organ','origin','system','pancancer']
dir_out = './figure4/'

dir_maf = '../maf_out/maf_cohorts_060121'
feature_type = 'histology'

# Read the significant gene dataframe after FDR calculation
feature_type = 'histology';syn_nsyn = 'syn';run = 'cohort_072221';threshold = 1;
df_syn = pd.read_csv(os.path.join(dir_out,feature_type+'.syn.df_all_forheatmap.'+run+'.'+str(threshold)+'.csv'),index_col = 0)
df_syn = df_syn.set_index('gene')

# For copy number variation
# Read cnv file
df_info = pd.read_csv('../anno_ref/proc_refs/PCAWG_sample_info.txt', sep = '\t')
df_cnv = pd.read_csv('../anno_ref/ICGC/consensus_cnv/consensus_CN.by_gene.170214.txt', sep = '\t')
df_cnv = df_cnv.set_index(['Gene Symbol', 'Locus ID', 'Cytoband'])

In [4]:
df_syn

,X,p,q,feature,exp.nonexp,FDR
gene,,,,,,
BCL2,0,0.000000,0.000000,Lymph-BNHL,exp,1.518254e-18
ITLN1,54504,0.000000,0.000000,CNS-PiloAstro,exp,1.518254e-18
XIRP2,109020,0.000000,0.000000,Skin-Melanoma,exp,1.518254e-18
SOX18,272521,0.000000,0.000000,Eso-AdenoCA,exp,1.518254e-18
SIGLEC15,345195,0.000000,0.000000,Panc-AdenoCA,exp,1.518254e-18
TP53I3,345196,0.000000,0.000000,Panc-AdenoCA,exp,1.518254e-18
CALR,399696,0.000000,0.000000,Uterus-AdenoCA,exp,1.518254e-18
PPWD1,417864,0.000000,0.000000,Head-SCC,exp,1.518254e-18
PURA,563208,0.000000,0.000000,Breast-AdenoCA,exp,1.518254e-18


In [15]:
def get_mut(gene):
    global df_syn
    histology = df_syn.loc[gene,'feature'] #histology type
    df_maf = pd.read_csv(os.path.join(dir_maf,feature_type, histology+'.csv'), sep = '\t')
    df_maf = df_maf.set_index(['Hugo_Symbol','Variant_Classification','Donor_ID' ])
    ldonor_all = df_maf.index.get_level_values('Donor_ID').unique().tolist()
    print(f'Total {len(ldonor_all)} patients')
    df_gene_mut = df_maf.loc[pd.IndexSlice[gene, 'Silent',:],:]### All patients' specific histology in this histology type
    ldonor = df_gene_mut.index.get_level_values('Donor_ID').unique().tolist()
    print(f'{len(ldonor)} patients have synonymous mutations, they are: {ldonor}')
    for muts in df_gene_mut['Genome_Change'].tolist():
        chromosome = muts.split(':')[0].split('.')[-1].split('chr')[-1]
        change = muts.split(':')[1]
        mutation = chromosome+':g.'+change
        print(mutation)

    return df_maf, df_gene_mut, ldonor


In [16]:
gene_name = 'CALR'
df_all, df_mut, donor_list = get_mut(gene_name)

Total 43 patients
1 patients have synonymous mutations, they are: ['DO41217']
19:g.13051179C>T
19:g.13051188A>C
19:g.13051197T>G


In [19]:
df_all.loc[pd.IndexSlice[gene_name, :,donor_list],'Genome_Change']

Hugo_Symbol  Variant_Classification  Donor_ID
CALR         Silent                  DO41217     g.chr19:13051179C>T
                                     DO41217     g.chr19:13051188A>C
                                     DO41217     g.chr19:13051197T>G
             Missense_Mutation       DO41217     g.chr19:13051215A>C
Name: Genome_Change, dtype: object

In [56]:
# Get tumor aliquot id
for donors in donor_list:
    idx = df_info[df_info['icgc_donor_id'] == donors].index
    aliquot_id = df_info.loc[idx, 'tumour_specimen_aliquot_id']
    cnv = df_cnv.loc[pd.IndexSlice[gene_name,:,:],aliquot_id]
cnv